# Assignment-3
#Video Enhancement

Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


# At first, clone the repository in your Mydrive

In [ ]:
!git clone

In [3]:
%cd /content/drive/MyDrive/Assignment-3

/content/drive/MyDrive/Assignment-3


In [ ]:
basePath = "content/drive/MyDrive/Assignment-3"
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

#installing all the required libraries

!pip install -r requirements.txt

%cd /content/drive/MyDrive/Assignment-3/GFPGAN-master

!python setup.py develop

print("Installation complete.")

In [2]:
%cd /content/drive/MyDrive/Assignment-3

/content/drive/MyDrive/Assignment-3


In [ ]:
import cv2
from tqdm import tqdm
from os import path
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips

filename = "test3.mp4" #@param{type:"string"}
outputPath = "/content/drive/MyDrive/Assignment-3/results"
inputVideoPath = outputPath+'/'+filename
unProcessedFramesFolderPath = outputPath+'/frames'
gfpganFolderName = "GFPGAN-master"

if not os.path.exists(unProcessedFramesFolderPath):
    os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

# for frameNumber in tqdm(range(numberOfFrames)):
#     _, image = vidcap.read()
#     cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4) + '.jpg'), image)

!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None

import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

# Get FPS of the original video for the writer
inputVideoPath = outputPath +"/"+ filename
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("The video is " + str(fps) + " FPS.")

batch = 0
batchSize = 500
processed_clips =[]

from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
    print("hi")
    img_array = []
    start, end = i, i + batchSize
    print("processing ", start, end, end="\r")
    for filename in tqdm(dir_list[start:end]):
        filename = restoredFramesPath + filename;
        img = cv2.imread(filename)
        if img is None:
            continue
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

    out = cv2.VideoWriter(processedVideoOutputPath + '/output_' + str(batch).zfill(4) + '.mp4',
                          cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    batch = batch + 1

    for i in range(len(img_array)):
        out.write(img_array[i])

    out.release()

    processed_clips.append(VideoFileClip(processedVideoOutputPath + f'/output_{str(batch-1).zfill(4)}.mp4'))

final_clip = concatenate_videoclips(processed_clips,method="compose")


# Get the audio from the original video
clip = VideoFileClip(inputVideoPath)
audio = clip.audio

# Combine the processed video and original audio

final_clip = final_clip.set_audio(audio)
# Save the final video
final_clip.write_videofile(outputPath + '/final_result.mp4', codec='libx264', audio_codec='aac')

from IPython.display import clear_output
# clear_output()

print("Video upscaling complete.")


Run this cell to play Enhanced videos, If it is not working you can download the "final_result.mp4" file in to you system from results folder

In [2]:
from IPython.display import display, HTML, Video

# Specify the path to the video file
video_path = "/content/drive/MyDrive/Assignment-3/results/final_result.mp4"

# Display the video
display(Video(video_path, width=400, height=300))

Run to remove all cache files

In [7]:
%rm /content/drive/MyDrive/Assignment-3/results/frames/*
%rm /content/drive/MyDrive/Assignment-3/results/restored_imgs/*


rm: cannot remove '/content/drive/MyDrive/Assignment-3/results/frames/*': No such file or directory
rm: cannot remove '/content/drive/MyDrive/Assignment-3/results/restored_imgs/*': No such file or directory


#Evaluation metric (PSNR)

Average Psnr values of given videos
1. Test1.mp4 - 35.616481395426824 dB
2. Test2.mp4 - 37.27242529287235 dB
3. Test3.mp4 - 37.27242529287235 dB

Run below cell to calculate average psnr values

In [4]:
import cv2
import numpy as np

def calculate_psnr(original, enhanced):
    """
    Calculate PSNR between two images.

    Parameters:
    - original: Original image
    - enhanced: Enhanced image

    Returns:
    - psnr: PSNR value
    """
    mse = np.mean((original - enhanced) ** 2)
    if mse == 0:
        return float('inf')  # PSNR is infinite if the images are identical
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_video_psnr(original_path, enhanced_path):
    """
    Calculate average PSNR for each frame in a video.

    Parameters:
    - original_path: Path to the original high-quality video
    - enhanced_path: Path to the enhanced video for comparison

    Returns:
    - average_psnr: Average PSNR across all frames
    """
    original_video = cv2.VideoCapture(original_path)
    enhanced_video = cv2.VideoCapture(enhanced_path)

    frame_count = int(original_video.get(cv2.CAP_PROP_FRAME_COUNT))
    total_psnr = 0.0

    for _ in range(frame_count):
        _, original_frame = original_video.read()
        _, enhanced_frame = enhanced_video.read()

        if original_frame is None or enhanced_frame is None:
            break

        # Resize frames to the same dimensions
        enhanced_frame = cv2.resize(enhanced_frame, (original_frame.shape[1], original_frame.shape[0]))

        original_frame = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)
        enhanced_frame = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2GRAY)

        psnr = calculate_psnr(original_frame, enhanced_frame)
        total_psnr += psnr

    original_video.release()
    enhanced_video.release()

    average_psnr = total_psnr / frame_count
    return average_psnr




write complete path of both original video and enhanced video

In [6]:
original_video_path = '/content/drive/MyDrive/Assignment-3/results/test3.mp4' #@param{type:"string"}
enhanced_video_path = '/content/drive/MyDrive/Assignment-3/results/video3_enhanced.mp4' #@param{type:"string"}

average_psnr_value = calculate_video_psnr(original_video_path, enhanced_video_path)
print(f'Average PSNR: {average_psnr_value} dB')


Average PSNR: 37.27242529287235 dB
